# Lecture G. Multiple Linear Regression using python.

Preliminary Definitions:   
[Fit a model](https://en.wikipedia.org/wiki/Goodness_of_fit)  
[Overfitting VS Underfitting](https://en.wikipedia.org/wiki/Overfitting)  
[Bias-Variance tradeoff (underfit VS overfit)](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff)   
High Bias => Underfit.  
High Variance = > Overfit


## Install necessary libraries with activated virtual env.
* Install from Windows Command Line, not from here!

* [Pingouin](https://pingouin-stats.org/) is new but funky and very helpful for beginners. Endless source of cuteness.  
Must read: [Pingouin guidelines.](https://pingouin-stats.org/guidelines.html)   

* [Statsmodels](https://www.statsmodels.org/stable/index.html) is more advanced, has more features. Has extensive [examples.](https://www.statsmodels.org/stable/examples/index.html)   

Understand what version 0.x.x means!

```
pip install statsmodels
pip install pingouin
pip install scikit-learn
```

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# from statsmodels.sandbox.regression.predstd import wls_prediction_std
import pingouin as pg
import statsmodels.api as sm

ModuleNotFoundError: No module named 'pingouin'

In [ ]:
# pandas set display output options
pd.set_option("display.precision", 3)

In [ ]:
# pandas set display output options
# Uncomment line below to see the error.

#pd.set_option("precision", 3)  #  This was working fine last year, now it is not.

# I found the correct syntax using:
# pd.set_option?

## Read and prepare the data

In [ ]:
# This is the path on my PC not yours!
# Learn how to set the proper path, please, please!
df = pd.read_excel("../data/calc_grades_determinants.xlsx")
# Notice that openpyxl is a dependency that pandas requires to read excel files.
# Although we don't need to import it, we have to install it.

    ModuleNotFoundError  
    ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

    So, with the virtual environment activated:  

```
pip install openpyxl  
```

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
# this step may be avoided, since we create X and y by numeric index
df.columns = df.columns.str.replace(" ", "_").str.lower()  # or str.replace(" ", "") # to replace blank space with nothing
df.columns

In [ ]:
# get column names and data type in each column
df.dtypes

In [ ]:
df.alg2_grade.unique()

In [ ]:
df.alg2_grade.unique()

In [ ]:
df.dtypes## Example of how to select some columns by HEADER name
df[["calc_hs", "act_math", "alg_place"]].head(2)

### Select the features columns X, not include gender and Y  
Array data strucure = indexed collection of data elements.   
This is an n-dimensional array, a matrix, conventionally denoted with capital letters.   
The feature matrix is denoted with X (usually).  
Sometimes, I prefer properly named subsets. E.g. we could call it ```all_X```

In [ ]:
# exclude gender and calc
# Can you try it using column labels?
X = df.iloc[:, :-2]
X.head(3)

In [ ]:
# 80rows, 6 cols, index NOT INCLUDED
X.shape

### Select dependent variable column (aka, target, predicted, ...)  
The predicted variable is a one dimensional array (1d-vector) conventionally denoted with small letters.  
Usually it is called as y, if more than one better use explicit naming.   
Different algorithms ask for different input.   
Be very careful with the data structure:
* type
* shape
* dimensions

In [ ]:
# this notation returns a pandas Series
y = df.iloc[:, -1]
y.head(3)

In [ ]:
type(y)

In [ ]:
y.shape

In [ ]:
np.ndim(y)

In [ ]:
# if you prefer this output
# this notation returns a pandas dataframe
# y = df.iloc[:, -1:]
df.iloc[:, -1:].head(3)

In [ ]:
type(df.iloc[:, -1:])

In [ ]:
# 80 rows, 1 column
df.iloc[:, -1:].shape

In [ ]:
# 2d array
np.ndim(df.iloc[:, -1:])

In [ ]:
# this is the last row as a dataframe, not the last column
df_last_row = df.iloc[[-1]]
df_last_row

In [ ]:
type(df_last_row)

In [ ]:
# this returns the last row as "pandas series type".
series_last_row = df.iloc[-1]
series_last_row

In [ ]:
type(series_last_row)

### Categorical Variables Encoding

In [ ]:
# create a new df for demonstration reason
df_enc = df.copy()

In [ ]:
df_enc["gender_code"] = df_enc["gender_code"].astype("category")

df_enc.dtypes

In [ ]:
# hot encode gender. drop_first=True, not used so set columns of X one by one. 
df_enc = pd.get_dummies(df_enc, prefix=["gender"], columns=["gender"])

In [ ]:
df_enc.columns

In [ ]:
df_enc.gender_F.values

## Examine the various variable correlation with each other.

### Correlation table

In [ ]:
df.corr(method='pearson')

### Correlation plots

In [ ]:
# set default size options, width, height
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
df_corr = df.corr()

sns.heatmap(
    df_corr, annot = True, cmap = 'Blues',
    xticklabels = df_corr.columns.values,
    yticklabels = df_corr.columns.values
);

plt.title('Calculus grades Heatmap', fontsize = 15);
plt.xticks(fontsize = 12);
plt.yticks(fontsize = 12);

In [ ]:
df.loc[(df.gender_code == 1), "gender"].unique()

In [ ]:
df[["gender_code", "gender"]].values[:3]

In [ ]:
# pairwise correlation comparison
sns.pairplot(
    data = df[[ "act_math", "alg_place", "gender_code", "calc"]],
    hue = 'gender_code' , palette = ['Violet', 'Blue']);

## Linear regression libraries

### Linear regression using pingouin

In [ ]:
# y was used as a Series
# just in case we go up and down
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

In [ ]:
X.head()

In [ ]:
lm_df = pg.linear_regression(X, y, add_intercept=True)
lm_df

In [ ]:
type(lm_df)

In [ ]:
# help(pg.linear_regression)

In [ ]:
lm_df[["names", 'coef']]

In [ ]:
lm_df.residuals_

In [ ]:
pd.Series(lm_df.residuals_).describe().round(2)

In [ ]:
# regression get output not as df
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

lm_not_df = pg.linear_regression(X, y, add_intercept=True, as_dataframe=False)
# lm_not_df

In [ ]:
lm_not_df['pred']

In [ ]:
pd.Series(lm_not_df["residuals"]).describe().round(2)

In [ ]:
# # If y is input as a dataframe data type
# # AssertionError: y must be one-dimensional.

# y = df.iloc[:, -1:]

# pg.linear_regression(X, y, add_intercept=True)

In [ ]:
# # regression using encoded column for gender
# lm_enc = pg.linear_regression(
#     df_enc[['calc_hs', 'act_math', 'alg_place', 'alg2_grade', 'hs_rank', 'gender_M']],
#     y, add_intercept=True)
# lm_enc

### Linear regression using statsmodels

In [ ]:
# add a constant intercept differently
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

In [ ]:
X.head()

In [ ]:
X = sm.add_constant(X)
X.head(2)

In [ ]:
ols_results = sm.OLS(y, X).fit()
print(ols_results.summary())

In [ ]:
type(ols_results)

In [ ]:
print(ols_results.summary2())

In [ ]:
# different output representation
ols_results.summary()

In [ ]:
ols_results.diagn

In [ ]:
ols_results.aic

In [ ]:
ols_results.tvalues

In [ ]:
# dir(ols_results)

In [ ]:
ols_results.pvalues

In [ ]:
ols_results.resid

In [ ]:
pd.Series(ols_results.resid).describe().round(2)

In [ ]:
# sum of squared residuals or RSS residual sum of squares
ols_results.ssr

In [ ]:
compare_pred_df = pd.DataFrame({'actual_y': y, 'y_hat':ols_results.fittedvalues})
compare_pred_df.head(10)

### Lineat Regression Plots

In [ ]:
sns.scatterplot(x="actual_y", y="y_hat", color="b", data=compare_pred_df);

In [ ]:
# takes arrays as arguments
sns.scatterplot(x=y, y=ols_results.fittedvalues, color="b");

In [ ]:
g = sns.lmplot(x="actual_y", y="y_hat", data=compare_pred_df)

In [ ]:
sns.residplot(x=ols_results.fittedvalues, y=ols_results.resid, lowess=True, color="g");

In [ ]:
sm.qqplot(ols_results.resid);
# plt.show()

In [ ]:
from scipy import stats

stats.probplot(ols_results.resid, plot=sns.mpl.pyplot);

In [ ]:
sns.displot(x=ols_results.resid, kind="kde");

In [ ]:
sns.displot(x=ols_results.resid, kde=True);

In [ ]:
sns.displot(x=ols_results.resid, kind="ecdf", rug=True);

In [ ]:
sns.displot(x=y, kind="ecdf",  rug=True);

In [ ]:
sns.displot(x=y, kind="kde");

In [ ]:
# # documentation on the function
# help(sm.OLS)

In [ ]:
# # documentation on the output
# help(ols_results)

### Remove non statistically significant factors

In [ ]:
# four features, mind the F-statistic
X = df[["calc_hs", "act_math", "alg2_grade", "alg_place"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()

### Best model I managed  to find   

* three features  
* F-statistic
* p-values 

In [ ]:
# three features, mind the F-statistic, the 

X = df[["calc_hs", "act_math", "alg2_grade"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()

In [ ]:
X = df[["calc_hs", "act_math", "alg2_grade"]]
lm = pg.linear_regression(X, y, add_intercept=True)
lm

In [ ]:
# three features, mind the F-statistic
# the p-value of act_math is not ok
X = df[["calc_hs", "act_math", "alg_place"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()

In [ ]:
# tho features, mind the F-statistic
X = df[["calc_hs", "alg_place"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()

In [ ]:
# tho features, mind the F-statistic
X = df[["calc_hs", "alg2_grade"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()

In [ ]:
# tho features, mind the F-statistic
X = df[["calc_hs", "act_math"]]
X = sm.add_constant(X)

ols_results = sm.OLS(y, X).fit()
ols_results.summary()